In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
import tweepy
from textblob import TextBlob

ModuleNotFoundError: No module named 'tweepy'

In [ ]:
consumer_key= 'consumer_key'
consumer_secret= 'consumer_secret'

access_token='access_token'
access_token_secret='access_token_secret'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
public_tweets = api.search('@realDonaldTrump')

In [4]:
data = {'text':[],'sentiment':[]}
data1 = {'text':[],'sentiment':[]}

In [5]:
for tweet in public_tweets:
    data['text'].append(tweet.text)

NameError: name 'public_tweets' is not defined

In [6]:
data['text']

[]

In [8]:
data1['text'] = pd.Series(data['text'])

In [9]:
data1['text'] = data1['text'].apply(lambda x: x.lower())
data1['text'] = data1['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', ' ', x)))

size = 2200
tokenizer = Tokenizer(num_words=size, split=' ')
tokenizer.fit_on_texts(data1['text'].values)
X2 = tokenizer.texts_to_sequences(data1['text'].values)
X2 = pad_sequences(X2,maxlen=19)

In [10]:
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split

In [11]:
data = pd.read_csv("Headline_Trainingdata.csv", sep=',', quotechar='"', header=0, usecols=['text', 'sentiment'])
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', ' ', x)))

size = 2200
tokenizer = Tokenizer(num_words=size, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [12]:
model = Sequential()
model.add(Embedding(size, 300, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.5))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
model.fit(X_train, Y_train, epochs=50, batch_size=32, verbose=2)

score, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print("Accuracy: %.2f" % accuracy)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 300)           660000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 19, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 19, 128)           219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 19, 64)            49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 19, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
__________

KeyboardInterrupt: 

In [ ]:
pred = model.predict_classes(X2)
pred

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
#to check the loading process
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model.h5")
print("Loaded model from disk")
